In [1]:

import pandas as pd
import json
import geopandas as gpd
import streamlit as st
import plotly.express as px
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))


plotly_mimetype+notebook


In [3]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [4]:
#read in csv file
df_nta = pd.read_csv(r'data\time_away_ntas_2020.csv')

#melt for animation formatl
list_date = df_nta.columns.tolist()
list_date = list_date[61:153] # march 1st to may 31st
# list_date = list_date[61:63] # march 1st to march 2nd
df = pd.melt(df_nta, id_vars=['NTA'],value_vars = list_date)
df = df.rename(columns={'NTA':'id'}) #rename so the join ids are the same
df['variable'] = df.variable.str.replace('-','/')
df['val_pct'] = df['value']*100 # convert it into percentage
df.tail(20)

,id,variable,value,val_pct
17276,QN72,5/31/20,0.021489,2.148925
17277,QN76,5/31/20,0.018945,1.894521
17278,SI01,5/31/20,0.016804,1.680358
17279,SI05,5/31/20,0.013588,1.358795
17280,SI07,5/31/20,0.015129,1.512859
17281,SI08,5/31/20,0.015398,1.539778
17282,SI11,5/31/20,0.017876,1.787649
17283,SI12,5/31/20,0.030886,3.088578
17284,SI14,5/31/20,0.020040,2.004008
17285,SI22,5/31/20,0.020266,2.026599


In [5]:
df['val_pct'].describe()

count    17296.000000
mean         3.279158
std          2.883630
min          0.000000
25%          1.853653
50%          2.584720
75%          3.595376
max         27.328431
Name: val_pct, dtype: float64

In [6]:
#read in geojson file
map_ntas = json.load(open(r'data\shapefiles\NTA map.geojson','r'))

#check properties
map_ntas['features'][0]['properties']

{'ntacode': 'BK43',
 'shape_area': '35799637.8103',
 'ntaname': 'Midwood',
 'shape_leng': '27996.5912736',
 'boroname': 'Brooklyn',
 'borocode': '3',
 'countyfips': '047'}

In [7]:
# add feature id to geojson file
for feature in map_ntas['features']:
    feature['id'] = feature['properties']['ntacode']

In [8]:
# check to make sure id is working
map_ntas['features'][0]['id']

'BK43'

In [9]:
nbhood = gpd.read_file(r'data\shapefiles\nynta.shp')
nbhood.drop(columns = ['BoroCode','CountyFIPS','Shape_Leng','Shape_Area','geometry'])
df = pd.merge(df, nbhood, left_on = 'id', right_on = 'NTACode', how = 'inner')
#df = df.drop(columns = ['NTACode'])
df = df.rename({'NTAName':'neighborhood', 'variable':'Date', 'val_pct':'percent_away'}, axis = 1)
df['neighborhood'] = df['neighborhood'].str.replace('-',' ') # replace - value with space
df.head()
# df.to_csv(r'data\pct_time_away.csv')

,id,Date,value,percent_away,BoroCode,BoroName,CountyFIPS,NTACode,neighborhood,Shape_Leng,Shape_Area,geometry
0,BK09,3/1/20,0.007788,0.778816,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
1,BK09,3/2/20,0.011220,1.122020,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
2,BK09,3/3/20,0.007194,0.719425,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
3,BK09,3/4/20,0.008557,0.855746,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
4,BK09,3/5/20,0.007979,0.797872,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."


In [10]:
#converting from df to GeoJSON
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [1]:
fig = px.choropleth_mapbox(df, geojson = map_ntas,
                   locations ='id',
                   color = 'percent_away',
                   hover_name = 'neighborhood',
                   animation_frame ='Date',
                   color_continuous_scale = "oranges",
                   opacity = 0.99,
                   labels={'percent_away':'% of Residents', 'id':'NTA Code'},
                   range_color = [0,30])
# fig.add_annotation(text = 'Percent of NYC Residents<br>Away from Home >22 hours<br>per Day by Neighborhood',
#                   align = 'left', x = 0.03, y = 0.95,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 21 ))
#fig.add_annotation(text = 'Residents Away from Home',
 #                 align = 'right', x = 0.08, y = 0.90,
  #                showarrow = False,
   #               bordercolor = None,
    #              bgcolor = None,
     #             opacity = 0.95,
      #            font = dict(family = "Droid Serif", color = 'black', size = 28 ))
# fig.add_annotation(text = 'March 1, 2020 to May 31st, 2020',
#                   align = 'right', x = 0.03, y = 0.82,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 17 ))
# fig.add_annotation(text = "Click 'Play' to run through the animation<br>Or drag the slider to view specific dates",
#                   align = 'left', x = 0.73, y = 0.02,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 12 ))
fig.update_layout(mapbox_zoom = 9.45,
                 mapbox_center = {'lat':40.7, 'lon':-73.95},
                 mapbox_style="carto-darkmatter",
#                  margin = {'r':1,'t':1,'l':1,'b':1},
                 width = 950,
                 height = 780,
       #          legend_title_text = '% of NTA Residents',
                 legend = dict(yanchor = "top", y = 0.6, xanchor = "left", x = 0.10))

#fig.show()
fig.write_html('pct_time_away.html', auto_open = True, auto_play = False, include_plotlyjs = 'cdn')
# with open('pct_time_away.html') as f:
#     f.write(fig.to_html(auto_play = False, include_plotlyjs = 'cdn'))

NameError: name 'px' is not defined

In [ ]:
# st.plotly_chart(fig)